In [20]:
import pandas as pd
import numpy as np
data = pd.read_csv("data_pre3.csv", encoding="gbk")#
data.head()

C:\Users\WH\AppData\Local\Temp\ipykernel_13228\400337828.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data_pre3.csv", encoding="gbk")#


,url,id,Lng,Lat,Cid,DOM,followers,totalPrice,price,square,...,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage,tradeyear,trademon,tradeday
0,https://bj.lianjia.com/chengjiao/101084782030....,101084782030,116.475489,40.019520,1111027376244,1464.0,106,415.0,31680,131.00,...,6,0.217,1.0,0.0,1.0,7,56021.0,2016,8,9
1,https://bj.lianjia.com/chengjiao/101086012217....,101086012217,116.453917,39.881534,1111027381879,903.0,126,575.0,43436,132.38,...,6,0.667,1.0,1.0,0.0,7,71539.0,2016,7,28
2,https://bj.lianjia.com/chengjiao/101086041636....,101086041636,116.561978,39.877145,1111040862969,1271.0,48,1030.0,52021,198.00,...,6,0.500,1.0,0.0,0.0,7,48160.0,2016,12,11
3,https://bj.lianjia.com/chengjiao/101086406841....,101086406841,116.438010,40.076114,1111043185817,965.0,138,297.5,22202,134.00,...,6,0.273,1.0,0.0,0.0,6,51238.0,2016,9,30
4,https://bj.lianjia.com/chengjiao/101086920653....,101086920653,116.428392,39.886229,1111027381174,927.0,286,392.0,48396,81.00,...,2,0.333,0.0,1.0,1.0,1,62588.0,2016,8,28


In [21]:
data_new=data.copy()
data_new["livingRoom"] = data_new["livingRoom"].apply(pd.to_numeric, errors='coerce')
data_new["drawingRoom"] = data_new["drawingRoom"].apply(pd.to_numeric, errors='coerce')
data_new["bathRoom"] = data_new["bathRoom"].apply(pd.to_numeric, errors='coerce')
for col in data_new.columns:
    print(col+' : '+ str(data_new[col].dtype))

url : object
id : object
Lng : float64
Lat : float64
Cid : int64
DOM : float64
followers : int64
totalPrice : float64
price : int64
square : float64
livingRoom : float64
drawingRoom : float64
kitchen : int64
bathRoom : float64
floor : float64
buildingType : float64
constructionTime : object
renovationCondition : int64
buildingStructure : int64
ladderRatio : float64
elevator : float64
fiveYearsProperty : float64
subway : float64
district : int64
communityAverage : float64
tradeyear : int64
trademon : int64
tradeday : int64


In [22]:
import sklearn.model_selection as ms
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor as XGBR
from sklearn.ensemble import GradientBoostingRegressor

In [23]:
rf_params = [{
    'max_depth': [30], # 5 10 15 20
    'n_estimators': [120, 130], # 10 30 50 150
    'max_features': [6, 8], #1 2 4 
}]   

xgb_params = [{
    'max_depth': [9],  # 2-15 
    'min_child_weight':[2],
    'booster':['gbtree', 'dart'],
    'eta':[0.2, 0.3, 0.4]    
}] 

svr_params = [
#     {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},   
#     {'kernel': ['poly'], 'C': [1], 'degree': [2, 3]},
#     {'kernel': ['rbf'], 'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001]}
    {'kernel': ['sigmoid'], 'C': [1, 10, 100, 1000]},   
    {'kernel': ['precomputed'], 'C': [1, 10, 100, 1000]},
]

dt_params =[{
    'max_depth':[10, 15, 20],  # 30, 40
    'splitter':['best'],  # 'random'
}]
    
gbr_params = [{
    'n_estimators': [500,100],
    'max_depth': [6,],
    'min_samples_split': [5,],
    'learning_rate': [0.01,],
    'loss': ['ls'],
}]
          
models = {
    'DT':ms.GridSearchCV(DecisionTreeRegressor(random_state=1), dt_params, cv=3),
    'RF': ms.GridSearchCV(RandomForestRegressor(random_state=1), rf_params, cv=3),
    'GBR': ms.GridSearchCV(GradientBoostingRegressor(random_state=1), gbr_params, cv=3),
    'XGBR': ms.GridSearchCV(XGBR(),xgb_params, cv=3),
#     'SVR':ms.GridSearchCV(SVR(),svr_params, cv=3),
}

In [24]:
def fit_models(x_train,y_train):
    mfit = {model: models[model].fit(x_train, y_train) for model in models.keys()}
    b_params = {model: models[model].best_params_ for model in models.keys()}
    print(b_params)
    b_score = {model: models[model].best_score_ for model in models.keys()}
    print(b_score)

In [25]:
answer = pd.DataFrame()
def derive_positions(x_test):
    for model in models.keys():
        answer['pre'+model] = models[model].predict(x_test)

In [26]:
data_new.drop(["url", "id", "price","constructionTime"],axis=1, inplace=True)
data_new.isnull().sum()
data_new.shape

(318851, 24)

In [27]:
x = data_new.copy()
x.drop(['totalPrice'],axis=1, inplace=True)
y = data_new['totalPrice']
x.shape,y.shape

((318851, 23), (318851,))

In [28]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
fit_models(x_train,y_train)
derive_positions(x_test)

c:\Users\WH\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
c:\Users\WH\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
c:\Users\WH\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
c:\Users\WH\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(
c:\Users\WH\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed

{'DT': {'max_depth': 15, 'splitter': 'best'}, 'RF': {'max_depth': 30, 'max_features': 8, 'n_estimators': 130}, 'GBR': {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 6, 'min_samples_split': 5, 'n_estimators': 500}, 'XGBR': {'booster': 'dart', 'eta': 0.2, 'max_depth': 9, 'min_child_weight': 2}}
{'DT': 0.8897773874799576, 'RF': 0.92846340813225, 'GBR': 0.923730202820472, 'XGBR': 0.9362669220278818}


In [29]:
from sklearn.metrics import mean_squared_error #MSE
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import r2_score#R 2
scores = pd.DataFrame({'Model': [], 'RMSE': [], 'R2': []})
effect = pd.DataFrame(columns = ['model','MSE','MAE','RMSE','R2'])
i = 0
for k,v in models.items():
    y_predict = answer['pre'+k]
    MSE = mean_squared_error(y_test,y_predict)
    MAE = mean_absolute_error(y_test,y_predict)
    RMSE = np.sqrt(mean_squared_error(y_test,y_predict))  
    R2 = r2_score(y_test,y_predict)
    effect.loc[i] = [k,MSE,MAE,RMSE,R2]
    i += 1
    print(f"MSE:{MSE}MAE:{MAE}RMSE:{RMSE}R2:{R2}")
effect

MSE:4787.20547066131MAE:35.70184847923158RMSE:69.18963412723983R2:0.9097481976787505
MSE:2696.6627560849306MAE:26.99244022923524RMSE:51.9294016534461R2:0.9491605957837376
MSE:3174.566889093497MAE:32.910330704266656RMSE:56.34329497902565R2:0.9401508071700813
MSE:2319.864800937384MAE:25.381231712032807RMSE:48.16497483584295R2:0.9562642588229449


,model,MSE,MAE,RMSE,R2
0,DT,4787.205471,35.701848,69.189634,0.909748
1,RF,2696.662756,26.992440,51.929402,0.949161
2,GBR,3174.566889,32.910331,56.343295,0.940151
3,XGBR,2319.864801,25.381232,48.164975,0.956264
